# Predicting Human Gait Cycles Using Accelerometer Data

## Background



## Libraries

In [1]:
import numpy as np
import pandas as pd

## Data Collection

I used the accelerometer on my iPhone to collect accleration data while I was walking.

An accelerometer is an electromechanical device used to measure acceleration forces. Such forces may be static, like the continuous force of gravity or, as is the case with many mobile devices, dynamic to sense movement or vibrations.


https://www.livescience.com/40102-accelerometers.html

Data Augmentation

## Data Preprocessing

## Visualization

## Modeling

3 models in parrallel

Regularization

## Future Work

1. Increase the dataset size

2. 

## References
